In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [2]:
neo4j_host = "bolt://52.206.105.229:7687"
neo4j_user = "neo4j"
neo4j_password = "ammonia-score-wrecks"

driver = GraphDatabase.driver(neo4j_host, auth=(neo4j_user, neo4j_password))


In [3]:
# Create full text index

create_full_text_index = """
CREATE FULLTEXT INDEX movie IF NOT EXISTS
FOR (n:Movie)
ON EACH [n.title]
"""

with driver.session() as session:
    session.run(create_full_text_index)

Download movie embedding file by using the following link and place the file in this directory.
Link: https://drive.google.com/file/d/1ELuq3k1LtZZ_qt7nxe-H6PyHlyl_p3wG/view?usp=sharing

In [4]:
# Import embeddings

embedding_df = pd.read_csv("movie_embeddings.csv")
params = embedding_df.values
step = 1000

with driver.session() as session:
    for i in range(0, len(embedding_df), step):
        print(f"Importing {i} batch of embeddings")
        batch = [{'id': str(x[0]), 'embedding': x[2]}
                 for x in params[i:i+step]]
        session.run("""
        UNWIND $data AS row
        MATCH (m:Movie {movieId:row.id})
        SET m.embedding = apoc.convert.fromJsonList(row.embedding);
        """, {'data': batch})

Importing 0 batch of embeddings
Importing 1000 batch of embeddings
Importing 2000 batch of embeddings
Importing 3000 batch of embeddings
Importing 4000 batch of embeddings
Importing 5000 batch of embeddings
Importing 6000 batch of embeddings
Importing 7000 batch of embeddings
Importing 8000 batch of embeddings
Importing 9000 batch of embeddings
